In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import os
import numpy as np

## Sample Code For Scraping The Data at Station

In [19]:
base_url = "https://enviinfo.cem.gov.vn"
endpoint = "/eip/default/call/json/get_indicators_have_data"

url = base_url + endpoint

payload = {
    'station_id': 28602553176253587650986727137, 
    'from_public': 1,
    'station_type': 4  
}

response = requests.post(url, data=payload)

if response.status_code == 200:
    print(response.json())
else:
    print("Lỗi:", response.status_code)


{'html': '<option value="CO" selected>CO</option><option value="NO2" selected>NO2</option><option value="O3" selected>O3</option><option value="PM-10" selected>PM-10</option><option value="PM-2-5" selected>PM-2-5</option><option value="SO2" selected>SO2</option>', 'success': True}


## API For Getting Data At Stations In 1900 Days

- AQI API: `/eip/default/call/json/get_aqi_data%3Fdate%3D1900%26aqi_type%3D0` for getting data in 1900 days
- Indicators API: `/eip/default/call/json/get_indicators_have_data` for getting the name of datatables columns
- Station Details API: `/eip/default/call/json/get_stations_by_province` for getting the `StationID`, `lat`, `lon`, and its `address`

In [20]:
# Configuration for the API request
api_base_url = "https://enviinfo.cem.gov.vn"
stations_endpoint = "/eip/default/call/json/get_stations_by_province"
complete_stations_url = f"{api_base_url}{stations_endpoint}"

# Detailed request payload for fetching station data
stations_payload = {
    'sEcho': 1,  # Request identifier for DataTables
    'iColumns': 3,  # Number of columns in the DataTable
    'sColumns': ',,',  # Column identifiers
    'iDisplayStart': 0,  # Start point in the data set (for pagination)
    'iDisplayLength': 77,  # Number of records to fetch
    'mDataProp_0': 0,  # Data property for the first column
    'sSearch_0': '',  # Search term for the first column
    'bRegex_0': False,  # Use regex for the first column search term?
    'bSearchable_0': True,  # Can the first column be searched?
    'mDataProp_1': 1,  # Data property for the second column
    'station_type': 4  # Filter for station type
}

# Perform the POST request to fetch station data by province
station_response = requests.post(complete_stations_url, data=stations_payload)

# Handling the response
if station_response.status_code == 200:
    # Saving the response data to a JSON file
    with open('province-stations/province_station_data.json', 'w') as file:
        json.dump(station_response.json(), file)
    print("Station data saved successfully.")
else:
    print(f"Error fetching station data: {station_response.status_code}")

Station data saved successfully.


In [69]:
# Define your base URL and specific endpoints
base_url = "https://enviinfo.cem.gov.vn"
endpoint_aqi_data = "/eip/default/call/json/get_aqi_data%3Fdate%3D1900%26aqi_type%3D0"
endpoint_indicators_data = "/eip/default/call/json/get_indicators_have_data"

# Construct complete URLs for the requests
url_aqi_data = f"{base_url}{endpoint_aqi_data}"
url_indicators_data = f"{base_url}{endpoint_indicators_data}"

# Load province data to get province IDs
with open("province-stations/province_station_data.json", "r") as file:
    province_data = json.load(file)

    province_ids = [BeautifulSoup(province[1], "html.parser").find("a")["data-id"] for province in province_data['aaData']]
    province_latitudes = [BeautifulSoup(province[1], "html.parser").find("a")["data-lat"] for province in province_data['aaData']]
    province_longitudes = [BeautifulSoup(province[1], "html.parser").find("a")["data-lon"] for province in province_data['aaData']]

# Initialize payloads for the requests
payload_aqi = {
    'sEcho': 1,
    'iColumns': 9,
    'sColumns': ',,,,,,,,',
    'iDisplayStart': 0,
    'iDisplayLength': 1900,
    'mDataProp_0': 0,
    'sSearch_0': '',
    'bRegex_0': False,
    'bSearchable_0': True,
}

payload_indicators = {
    'station_id': 0,  
    'from_public': 1,
    'station_type': 4,
}

# Process each province
for province_id in province_ids[70:77]:
    # Update payloads with the current province ID
    payload_aqi["station_id"] = province_id
    payload_indicators["station_id"] = province_id

    # Fetch available indicators for the current province
    response_indicators = requests.post(url_indicators_data, data=payload_indicators)
    if response_indicators.status_code == 200:
        indicator_html = response_indicators.json().get('html', '')
        soup = BeautifulSoup(indicator_html, 'html.parser')
        selected_indicators = [option.text for option in soup.find_all('option', selected=True)]
        
        # Fetch AQI data for the current province
        response_aqi = requests.post(url_aqi_data, data=payload_aqi)
        if response_aqi.status_code == 200:
            data = response_aqi.json()
            # Assign the new column names including the indicators
            data['aoColumns'] = ['ID', 'Date', 'AQI'] + selected_indicators

            # Insert 2 columns to store the (province_lattitude, province_longitude)
            data['aoColumns'].insert(1, 'Province Latitude')
            data['aoColumns'].insert(2, 'Province Longitude')

            # Insert the province latitude and longitude values ignoring the ID and Date columns
            data['aaData'] = [[*row[:1], province_latitudes[province_ids.index(province_id)], province_longitudes[province_ids.index(province_id)], *row[1:]] for row in data['aaData']]

            # Save the updated data to a file
            with open(f"data-province/test-{province_id}.json", "w") as file:
                json.dump(data, file)
            # print(f"Data for province {province_id} saved successfully.")
        else:
            print(f"Error fetching AQI data for province {province_id}: {response_aqi.status_code}")
    else:
        print(f"Error fetching indicators for province {province_id}: {response_indicators.status_code}")

In [70]:
with open("province-stations/province_station_data.json", "r") as f:
    province_data = json.load(f)
    province_names = []
    province_ids = []
    for province in province_data['aaData']:
        html_snippet = province[1]
        soup = BeautifulSoup(html_snippet, "html.parser")
        province_id = soup.find("a")["data-id"]
        province_name = soup.find("a").get_text()

        province_name = province_name.replace(":", "").replace("/", "-").replace("\\", "-")
        
        # Find the file with their data-id then change to the province name if exists, ifnot, ignore
        try:
            with open(f"data-province/test-{province_id}.json", "r") as file:
                data = json.load(file)
                with open(f"data-provinces-name/{province_name}.json", "w") as new_file:
                    json.dump(data, new_file)
        except FileNotFoundError:
            print(f"File with data-id {province_id} not found.")
        except Exception as e:
            print(f"Error processing data for province {province_name}: {e}")

### Preprocessing for HCM

In [46]:
HCM_station_data = {
  '1' : {
    'Address': 'DHQG, Linh Trung, Thủ Đức',
    'Province': 'TP HCM',
    'Province Latitude': '10.86994333',
    'Province Longitude': '106.7960143'
  },
  '2' : {
    'Address': '20, Nguyễn Trọng Trí, An Lạc, Bình Tân',
    'Province': 'TP HCM',
    'Province Latitude': '10.74097081',
    'Province Longitude': '106.6204143'
  },
  '3' : {
    'Address': 'KCN Tân Bình, Tây Thạnh, Tân Phú',
    'Province': 'TP HCM',
    'Province Latitude': '10.81621227',	
    'Province Longitude': '106.6204143'
  },
  '4' : {
    'Address': '49, Thanh Đa, P.27, Bình Thạnh',
    'Province': 'TP HCM',
    'Province Latitude': '10.81584553',	
    'Province Longitude': '106.7174282'
  },
  '5' : {
    'Address': '268, Nguyễn Đình Chiểu, P.6, Q.3',
    'Province': 'TP HCM',
    'Province Latitude': '10.77636612',		
    'Province Longitude': '106.6878094'
  },
  '6' : {
    'Address': 'MM18, Trường Sơn, P.14, Q.10',
    'Province': 'TP HCM',
    'Province Latitude': '10.78047163',	
    'Province Longitude': '106.6594579'
  }
}

# Read the file temp_data/data/aqi_hcm_2021_2022.csv
df_hcm = pd.read_csv("temp_data/data/aqi_hcm_2021_2022.csv")
# Convert the Station_No base on the HCM_station_data
# Add the Province, Province Latitude, Province Longitude, Address columns
df_hcm["Station_No"] = df_hcm["Station_No"].astype(str)
df_hcm["Province"] = df_hcm["Station_No"].map(lambda x: HCM_station_data[x]["Province"])
df_hcm["Province Latitude"] = df_hcm["Station_No"].map(lambda x: HCM_station_data[x]["Province Latitude"])
df_hcm["Province Longitude"] = df_hcm["Station_No"].map(lambda x: HCM_station_data[x]["Province Longitude"])
df_hcm["Address"] = df_hcm["Station_No"].map(lambda x: HCM_station_data[x]["Address"])

# Rename date to Date
df_hcm.rename(columns={"date": "Date"}, inplace=True)

# Convert the Date column from 31-12-2020 00:00:00 to 2021-01-01
df_hcm["Date"] = pd.to_datetime(df_hcm["Date"], format="%d-%m-%Y %H:%M").dt.strftime("%Y-%m-%d")

# Re-order the columns
df_hcm = df_hcm[["Station_No", "Province", "Address", "Province Latitude", "Province Longitude", "Date", "TSP" ,"PM2.5", "O3", "CO", "NO2", "SO2","Temperature", "Humidity"]]

# Save the updated DataFrame to a new CSV file
# df_hcm.to_csv("aqi_hcm_2021_2022_updated.csv", index=False)

In [47]:
# Print missing values of df_hcm
print(df_hcm.isnull().sum())

Station_No                0
Province                  0
Address                   0
Province Latitude         0
Province Longitude        0
Date                      0
TSP                      60
PM2.5                     0
O3                    10610
CO                     9065
NO2                    5666
SO2                   11006
Temperature            4437
Humidity               4432
dtype: int64


In [48]:
# Filling missing value with the mean of the column of each station No
for station_no in df_hcm["Station_No"].unique():
    for column in df_hcm.columns[6:]:
        df_hcm.loc[df_hcm["Station_No"] == station_no, column] = df_hcm.loc[df_hcm["Station_No"] == station_no, column].fillna(df_hcm.loc[df_hcm["Station_No"] == station_no, column].mean())

In [49]:
# Print missing values of df_hcm after filling missing values
print(df_hcm.isnull().sum())

Station_No            0
Province              0
Address               0
Province Latitude     0
Province Longitude    0
Date                  0
TSP                   0
PM2.5                 0
O3                    0
CO                    0
NO2                   0
SO2                   0
Temperature           0
Humidity              0
dtype: int64


### Calculate AQI

In [50]:
# Define breakpoints and AQI levels
breakpoints = {
    'O3': [(0, 54), (55, 70), (71, 85), (86, 105), (106, 200)],
    'PM2.5': [(0.0, 12.0), (12.1, 35.4), (35.5, 55.4), (55.5, 150.4), (150.5, 250.4), (250.5, 350.4), (350.5, 500.4)],
    'CO': [(0.0, 4.4), (4.5, 9.4), (9.5, 12.4), (12.5, 15.4), (15.5, 30.4), (30.5, 40.4), (40.5, 50.4)],
    'SO2': [(0, 35), (36, 75), (76, 185), (186, 304)],
    'NO2': [(0, 53), (54, 100), (101, 360), (361, 649), (650, 1249), (1250, 1649), (1650, 2049)]
}
aqi_levels = [(0, 50), (51, 100), (101, 150), (151, 200),
              (201, 300), (301, 400), (401, 500)]

def calculate_aqi(parameter, value, breakpoints):
    for index, (low, high) in enumerate(breakpoints):
        if low <= value <= high:
            aqi = round(((aqi_levels[index][1] - aqi_levels[index][0]) / (high - low)) * (value - low) + aqi_levels[index][0])
            return aqi
    return 0  # Return a default value if outside any breakpoint range

# Calculate total AQI for each row
df_hcm["AQI"] = df_hcm.apply(lambda x: max([calculate_aqi(param, x[param], breakpoints[param]) for param in ["O3", "PM2.5", "CO", "SO2", "NO2"] if x[param] is not None]), axis=1)

# Insert new columns PM-10 based on the TSP with the formula PM-10 = TSP * 0.8
df_hcm["PM-10"] = df_hcm["TSP"] * 0.8

# Rename 'PM2.5' to 'PM-2.5'
df_hcm.rename(columns={"PM2.5": "PM-2.5"}, inplace=True)

# Save the updated DataFrame to a new CSV file
# df_hcm.to_csv("aqi_hcm_2021_2022_updated.csv", index=False)

# Drop columns TSP, Station_No, Temperature, Humidity
df_hcm.drop(columns=["TSP", "Station_No", "Temperature", "Humidity"], inplace=True)


In [51]:
df_hcm.to_csv("final-data/aqi_hcm_2021_2022.csv", index=False)

## Pre-processing for All Provinces

In [52]:
# Initialize an empty DataFrame with predefined columns
df_columns = ["Province", "Province Latitude", "Province Longitude", "Date", "AQI", "CO", "NO2", "O3", "PM-10", "PM-2-5", "SO2"]
df = pd.DataFrame(columns=df_columns)

# Path to the folder containing the data files
folder_path = "data-provinces-name"

# Define a function to handle missing or "-" values
def replace_missing(value):
    if value == "-":
        return np.nan
    return value

# Process each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Extract province name from the filename
            province_name = filename[:-5]  # Assuming '.json' is the extension

            # Assuming 'aoColumns' contains column names directly
            # 'aaData' is the list of data rows
            file_columns = data.get("aoColumns", [])
            aqi_data = data.get("aaData", [])

            # Prepare data for DataFrame creation
            temp_data = []
            for row in aqi_data:
                # Replace "-" with np.nan and ensure the length matches file_columns
                row_data = [replace_missing(item) for item in row] + [np.nan] * (len(file_columns) - len(row))
                temp_data.append([province_name] + row_data[:len(file_columns)])

            # Create DataFrame for the current file
            temp_df = pd.DataFrame(temp_data, columns=["Province"] + file_columns)

            # Ensure all predefined columns are present, adding missing ones with np.nan
            for column in df_columns:
                if column not in temp_df.columns:
                    temp_df[column] = np.nan

            # Concatenate with the main DataFrame
            df = pd.concat([df, temp_df[df_columns]], ignore_index=True)

# Convert numeric columns to float, handling errors (like conversion failures) by ignoring them
numeric_columns = ["AQI", "CO", "NO2", "O3", "PM-10", "PM-2-5", "SO2"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

# Convert '%d/%m/%Y' date format to 'yyyy-mm-dd'
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

# Save the combined DataFrame to CSV
df.to_csv("combined_data.csv", index=False)

In [53]:
# Read the combined data from the CSV file
df = pd.read_csv("combined_data.csv")

# Convert the name of 'Province' column to 'Address'
df.rename(columns={"Province": "Address"}, inplace=True)

# Conver the name of 'PM-2-5' column to 'PM-2.5'
df.rename(columns={"PM-2-5": "PM-2.5"}, inplace=True)

df["Address"] = df["Address"].str.replace("Thừa Thiên Huế", "TT Huế")
df["Address"] = df["Address"].str.replace("HCM", "TP HCM")

# Separate the 'Address' column into 'Province' and 'Address' columns
# The Province will get 2 first words and the Address will get the rest
df["Province"] = df["Address"].str.split(" ").str[:2].str.join(" ")
df["Address"] = df["Address"].str.split(" ").str[2:].str.join(" ")

# Re-order the columns
df = df[["Province", "Address", "Province Latitude", "Province Longitude", "Date", "AQI", "CO", "NO2", "O3", "PM-10", "PM-2.5", "SO2"]]

# Combine the df_hcm and df which correspond to the same columns
df = pd.concat([df, df_hcm], ignore_index=True)

# Save the updated DataFrame to a new CSV file
df.to_csv("combined_data_updated.csv", index=False)

In [54]:
# Show the distinct value of Province column
print(df["Province"].unique())
print("Provinces count: ", df["Province"].nunique())

['Thanh Hoá' 'Hưng Yên' 'Hà Nam' 'Bắc Ninh' 'Thái Bình' 'Vũng Tàu'
 'Trà Vinh' 'Quảng Ngãi' 'Hà Nội' 'Quảng Ninh' 'TT Huế' 'Gia Lai'
 'Hậu Giang' 'Long An' 'Bắc Giang' 'Đà Nẵng' 'Bình Phước' 'Nghệ An'
 'TP HCM' 'Cao Bằng' 'Vĩnh Long' 'Hải Dương' 'Bình Dương' 'Ninh Thuận'
 'Lâm Đồng' 'Phú Thọ' 'Quảng Bình' 'Gia lai' 'Bình Định' 'Hà Tĩnh'
 'Khánh Hòa' 'Quảng Nam' 'Lào Cai' 'Lạng Sơn']
Provinces count:  34


In [55]:
# Print the properties of the DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108322 entries, 0 to 108321
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Province            108322 non-null  object 
 1   Address             108322 non-null  object 
 2   Province Latitude   108322 non-null  object 
 3   Province Longitude  108322 non-null  object 
 4   Date                108322 non-null  object 
 5   AQI                 108322 non-null  int64  
 6   CO                  96980 non-null   float64
 7   NO2                 97118 non-null   float64
 8   O3                  96735 non-null   float64
 9   PM-10               105154 non-null  float64
 10  PM-2.5              104623 non-null  float64
 11  SO2                 98812 non-null   float64
dtypes: float64(6), int64(1), object(5)
memory usage: 9.9+ MB
None


In [56]:
# Count the number of missing values in each column
print(df.isnull().sum())

Province                  0
Address                   0
Province Latitude         0
Province Longitude        0
Date                      0
AQI                       0
CO                    11342
NO2                   11204
O3                    11587
PM-10                  3168
PM-2.5                 3699
SO2                    9510
dtype: int64


### Fill missing value

In [57]:
df["Date"] = pd.to_datetime(df["Date"])
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year

def fill_by_same_month_year(group):
    """Fill missing values with the mean of the same month in the same year of the province."""
    numeric_cols = group.select_dtypes(include=[np.number]).columns.difference(['Year', 'Month'])
    for col in numeric_cols:
        if col not in ['Province', 'Date', 'Year']:
            group[col] = group[col].fillna(group[col].mean())
    return group

df = fill_by_same_month_year(df)

df.drop(columns=["Year", "Month"], inplace=True)

In [58]:
# Count lines has year before 2020 and after 2023
print(df[(df["Date"] < "2020-01-01") | (df["Date"] > "2023-12-31")].shape[0])

# Drop
df = df[(df["Date"] >= "2020-01-01") & (df["Date"] <= "2023-12-31")]

9274


In [59]:
print(df.isnull().sum())

Province              0
Address               0
Province Latitude     0
Province Longitude    0
Date                  0
AQI                   0
CO                    0
NO2                   0
O3                    0
PM-10                 0
PM-2.5                0
SO2                   0
dtype: int64


In [60]:
# save this updated DataFrame to a new CSV file
df.to_csv("final-data/aqi_vietnam_2020_2023.csv", index=False)

In [74]:
# # Read csv file temp_data/data/aqi_vietnam_2021.csv
df_vn_21 = pd.read_csv("temp_data/data/aqi_vietnam_2021.csv")

# Drop columns _id, Mã trạm, Url, Múi giờ, Trạng thái cảnh báo, Mức độ cảnh báo
df_vn_21.drop(columns=["_id", "Mã trạm", "Url", "Múi giờ", "Trạng thái cảnh báo", "Mức độ cảnh báo"], inplace=True)

In [75]:
df_vn_21.head()

,Chỉ số AQI,Vị trí,Tên trạm,Chất ô nhiễm chủ đạo,CO,Sương,Độ ẩm,NO2,O3,Áp suất,Bụi PM10,Bụi PM2.5,SO2,Nhiệt độ,Tốc độ gió,Thời gian cập nhật
0,102.00,"10.782978,106.700711","Ho Chi Minh City US Consulate, Vietnam (Lãnh s...",pm25,-,0.0,83.0,-,-,1009.0,0.00,102.00,-,27.0,3.6,2021-01-21T19:00:00
1,221.00,"21.0811211,105.8180306","United Nations International School of Hanoi, ...",aqi,-,0.0,77.0,-,-,1015.0,0.00,-,-,18.0,1.5,2021-01-21T20:00:00
2,281.00,"21.0215063,105.8188748","Hanoi US Embassy, Vietnam (Đại sứ quán Mỹ, Hà ...",pm25,0.30,0.0,77.0,8.70,0.90,1015.0,0.00,281.00,2.10,18.0,1.5,2021-01-21T20:00:00
3,36.00,"13.998599,107.996482","Gia Lai/phường Thống Nhất - Pleiku, Vietnam",pm25,3.00,0.0,60.2,6.00,9.00,923.7,29.00,36.00,1.00,24.3,1.0,2021-01-21T20:00:00
4,68.00,"16.46226,107.596351","Thừa Thiên Huế/83 Hùng Vương, Vietnam",pm25,2.00,0.0,88.0,-,-,1015.0,52.00,68.00,-,21.0,1.0,2021-01-21T19:00:00


In [76]:
# Separate the column Vị trí to Province Latitude and Province Longitude
df_vn_21["Province Latitude"] = df_vn_21["Vị trí"].str.extract(r"(\d+\.\d+),")
df_vn_21["Province Longitude"] = df_vn_21["Vị trí"].str.extract(r",(\d+\.\d+)")

In [77]:
df_vn_21.drop(columns=["Vị trí"], inplace=True)

In [85]:
# Convert the Date column from 2021-01-21T19:00:00 to 2021-01-01
df_vn_21["Thời gian cập nhật"] = pd.to_datetime(df_vn_21["Thời gian cập nhật"], format="%Y-%m-%dT%H:%M:%S").dt.strftime("%Y-%m-%d")
# Rename from Thời gian cập nhật to Date
df_vn_21.rename(columns={"Thời gian cập nhật": "Date"}, inplace=True)

In [86]:
df_vn_21.head()

,Chỉ số AQI,Chất ô nhiễm chủ đạo,CO,Sương,Độ ẩm,NO2,O3,Áp suất,Bụi PM10,Bụi PM2.5,SO2,Nhiệt độ,Tốc độ gió,Date,Province Latitude,Province Longitude,Province,Address
0,102.00,pm25,-,0.0,83.0,-,-,1009.0,0.00,102.00,-,27.0,3.6,2021-01-21,10.782978,106.700711,TP HCM,"US Consulate, Vietnam (Lãnh sự quán Hoa Kỳ, Hồ..."
1,221.00,aqi,-,0.0,77.0,-,-,1015.0,0.00,-,-,18.0,1.5,2021-01-21,21.0811211,105.8180306,Hà Nội,"United Nations International School of , Vietn..."
2,281.00,pm25,0.30,0.0,77.0,8.70,0.90,1015.0,0.00,281.00,2.10,18.0,1.5,2021-01-21,21.0215063,105.8188748,Hà Nội,"US Embassy, Vietnam (Đại sứ quán Mỹ, Hà Nội)"
3,36.00,pm25,3.00,0.0,60.2,6.00,9.00,923.7,29.00,36.00,1.00,24.3,1.0,2021-01-21,13.998599,107.996482,Gia Lai,"phường Thống Nhất - Pleiku, Vietnam"
4,68.00,pm25,2.00,0.0,88.0,-,-,1015.0,52.00,68.00,-,21.0,1.0,2021-01-21,16.46226,107.596351,Thừa Thiên Huế,"83 Hùng Vương, Vietnam"


In [80]:
import pandas as pd

def format_station_name(station_name):
    # Replace "Ho Chi Minh" with "TP HCM" and split the remaining part for province and address
    if "Ho Chi Minh City" in station_name:
        province = "TP HCM"
        address = station_name.replace("Ho Chi Minh City", "").strip()
    
    # Replace "Hanoi" with "Hà Nội" and split the remaining part for province and address
    elif "Hanoi" in station_name:
        province = "Hà Nội"
        address = station_name.replace("Hanoi", "").strip()
    
    # Split by '/' and format each part
    elif "/" in station_name:
        parts = station_name.split('/')
        province = parts[0].strip()
        address = parts[1].strip()
    
    # Split by ',' and format
    elif "," in station_name:
        parts = station_name.split(',', 1)
        province = parts[0].strip()
        address = parts[1].strip()
    
    # Split by space and format
    else:
        parts = station_name.split(' ', 1)
        province = parts[0].strip()
        address = parts[1].strip()
        # Format province
        if province == "US":
            province = "US Embassy"
        elif province == "UN":
            province = "United Nations"

    return province, address

def separate_and_format(df):
    df[['Province', 'Address']] = df['Tên trạm'].apply(lambda x: pd.Series(format_station_name(x)))
    return df


df_vn_21 = separate_and_format(df_vn_21)



In [30]:
# save to final-data/aqi_vietnam_2021.csv
df_vn_21.to_csv("final-data/aqi_vietnam_2021.csv", index=False)

In [ ]:
# Drop columns 'Tên trạm'
df_vn_21.drop(columns=["Tên trạm"], inplace=True)

In [88]:
# Rename Chỉ số AQI to AQI
df_vn_21.rename(columns={"Chỉ số AQI": "AQI"}, inplace=True)

# Rename Chất ô nhiễm chủ đạo to Primary Pollutant
df_vn_21.rename(columns={"Chất ô nhiễm chủ đạo": "Primary Pollutant"}, inplace=True)

# Rename Sương to Fog
df_vn_21.rename(columns={"Sương": "Fog"}, inplace=True)

# Rename Độ ẩm to Humidity
df_vn_21.rename(columns={"Độ ẩm": "Humidity"}, inplace=True)

# Rename Nhiệt độ to Temperature
df_vn_21.rename(columns={"Nhiệt độ": "Temperature"}, inplace=True)

# Rename Tốc độ gió to Wind Speed
df_vn_21.rename(columns={"Tốc độ gió": "Wind Speed"}, inplace=True)

# Rename Áp suất to Pressure
df_vn_21.rename(columns={"Áp suất": "Pressure"}, inplace=True)

# Rename Bụi PM10 to PM-10
df_vn_21.rename(columns={"Bụi PM10": "PM-10"}, inplace=True)

# Rename Bụi PM2.5 to PM-2.5
df_vn_21.rename(columns={"Bụi PM2.5": "PM-2.5"}, inplace=True)

In [90]:
# Reorder the columns
df_vn_21 = df_vn_21[["Province", "Address", "Province Latitude", "Province Longitude", "Date", "AQI", "CO", "NO2", "O3", "PM-10", "PM-2.5", "SO2", "Primary Pollutant", "Fog", "Humidity", "Temperature", "Wind Speed", "Pressure"]]

In [110]:
# Convert value - to np.nan
df_vn_21.replace('-', np.nan, inplace=True)

# Print the missing values of df_vn_21
print(df_vn_21.isnull().sum())

Province                0
Address                 0
Province Latitude       0
Province Longitude      0
Date                    0
AQI                   279
CO                    326
NO2                   308
O3                    275
PM-10                 380
PM-2.5                294
SO2                   327
Primary Pollutant     471
Fog                     0
Humidity                0
Temperature             0
Wind Speed              0
Pressure                0
dtype: int64


In [15]:
# Read from csv 
df_vn_21 = pd.read_csv("final-data/aqi_vietnam_2021.csv")

In [106]:
# Fill missing values with the mean of the column of each province

# Separate month from the Date column
df_vn_21["Month"] = pd.to_datetime(df_vn_21["Date"]).dt.month

# Fill missing values with the mean of the column of each province


def fill_by_same_month(group):
    """Fill missing values with the mean of the same month of the province."""
    numeric_cols = group.select_dtypes(include=[np.number]).columns.difference(['Month'])
    for col in numeric_cols:
        if col not in ['Province', 'Date', 'Month']:
            group[col] = group[col].fillna(group[col].mean())
    return group

df_vn_21 = df_vn_21.groupby(["Province", "Month"]).apply(fill_by_same_month)

df_vn_21.drop(columns=["Month"], inplace=True)

# Print the missing values of df_vn_21 after filling missing values
print(df_vn_21.isnull().sum())

Province                0
Address                 0
Province Latitude       0
Province Longitude      0
Date                    0
AQI                   279
CO                    326
NO2                   308
O3                    275
PM-10                 380
PM-2.5                294
SO2                   327
Primary Pollutant     471
Fog                     0
Humidity                0
Temperature             0
Wind Speed              0
Pressure                0
dtype: int64


In [111]:
def fill_by_same_province(group): 
    """Fill missing values with the mean of the column of each province."""
    numeric_cols = group.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col not in ['Province', 'Date']:
            group[col] = group[col].fillna(group[col].mean())
    return group

df_vn_21 = df_vn_21.groupby("Province").apply(fill_by_same_province)

# Print the missing values of df_vn_21 after filling missing values
print(df_vn_21.isnull().sum())

Province                0
Address                 0
Province Latitude       0
Province Longitude      0
Date                    0
AQI                     0
CO                    326
NO2                   218
O3                    218
PM-10                 108
PM-2.5                108
SO2                   327
Primary Pollutant     471
Fog                     0
Humidity                0
Temperature             0
Wind Speed              0
Pressure                0
dtype: int64


In [18]:
df = pd.read_csv("final-data/aqi_vietnam_2020_2023.csv")

# For the missing value in df_vn_21, fill with the mean of the column of each province of df if exists column in df
for province in df_vn_21["Province"].unique():
    for column in df_vn_21.columns[5:]:
        if column in df.columns:
            df_vn_21.loc[df_vn_21["Province"] == province, column] = df_vn_21.loc[df_vn_21["Province"] == province, column].fillna(df.loc[df["Province"] == province, column].mean())

# Print the missing values of df_vn_21 after filling missing values
print(df_vn_21.isnull().sum())

Province                0
Address                 0
Province Latitude       0
Province Longitude      0
Date                    0
AQI                     0
CO                      0
NO2                     0
O3                      0
PM-10                 108
PM-2.5                108
SO2                     0
Primary Pollutant     471
Fog                     0
Humidity                0
Temperature             0
Wind Speed              0
Pressure                0
dtype: int64


In [24]:
# Fill missing values in Nha Trang in df_vn_21 with the mean of the column of Khánh Hoà in df if exists column in df

for column in df_vn_21.columns[5:]:
    if column in df.columns:
        df_vn_21.loc[df_vn_21["Province"] == "Nha Trang", column] = df_vn_21.loc[df_vn_21["Province"] == "Nha Trang", column].fillna(df.loc[df["Province"] == "Khánh Hòa", column].mean())

# Print the missing values of df_vn_21 after filling missing values
print(df_vn_21.isnull().sum())

Province                0
Address                 0
Province Latitude       0
Province Longitude      0
Date                    0
AQI                     0
CO                      0
NO2                     0
O3                      0
PM-10                   0
PM-2.5                  0
SO2                     0
Primary Pollutant     471
Fog                     0
Humidity                0
Temperature             0
Wind Speed              0
Pressure                0
dtype: int64


In [25]:
# Primary Pollutant is the [CO, NO2, O3, PM-10, PM-2.5, SO2] which has the highest value
df_vn_21["Primary Pollutant"] = df_vn_21[["CO", "NO2", "O3", "PM-10", "PM-2.5", "SO2"]].idxmax(axis=1)

In [29]:
# Calc the AQI for each row

# Define breakpoints and AQI levels
breakpoints = {
    'O3': [(0, 54), (55, 70), (71, 85), (86, 105), (106, 200)],
    'PM-2.5': [(0.0, 12.0), (12.1, 35.4), (35.5, 55.4), (55.5, 150.4), (150.5, 250.4), (250.5, 350.4), (350.5, 500.4)],
    'PM-10': [(0.0, 54.0), (55.0, 154.0), (155.0, 254.0), (255.0, 354.0), (355.0, 424.0), (425.0, 504.0), (505.0, 604.0)],
    'CO': [(0.0, 4.4), (4.5, 9.4), (9.5, 12.4), (12.5, 15.4), (15.5, 30.4), (30.5, 40.4), (40.5, 50.4)],
    'SO2': [(0, 35), (36, 75), (76, 185), (186, 304)],
    'NO2': [(0, 53), (54, 100), (101, 360), (361, 649), (650, 1249), (1250, 1649), (1650, 2049)]
}
aqi_levels = [(0, 50), (51, 100), (101, 150), (151, 200),
              (201, 300), (301, 400), (401, 500)]

def calculate_aqi(parameter, value, breakpoints):
    for index, (low, high) in enumerate(breakpoints):
        if low <= value <= high:
            aqi = round(((aqi_levels[index][1] - aqi_levels[index][0]) / (high - low)) * (value - low) + aqi_levels[index][0])
            return aqi
    return 0  # Return a default value if outside any breakpoint range

df_vn_21["AQI"] = df_vn_21.apply(lambda x: max([calculate_aqi(param, x[param], breakpoints[param]) for param in ["O3", "PM-2.5", "CO", "SO2", "NO2", "PM-10"] if x[param] is not None]), axis=1)

In [33]:
df_vn_20 = pd.read_csv("temp_data/data/aqi_vietnam_2020.csv")

# Drop columns '_id'
df_vn_20.drop(columns=["_id"], inplace=True)

# Reformat Ngày from 2020-10-30T00:00:00 to 2020-10-30
df_vn_20["Date"] = pd.to_datetime(df_vn_20["Ngày"]).dt.strftime("%Y-%m-%d")

# Drop column Ngày
df_vn_20.drop(columns=["Ngày"], inplace=True)

In [34]:
df_vn_20.head()

,Quốc gia,Thành phố,Phân loại,Count,Min,Max,Meidan,Variance,Date
0,VN,Hạ Long,precipitation,3,0.2,99.3,0.2,32736.0,2020-10-30
1,VN,Hạ Long,precipitation,3,99.0,99.0,99.0,0.0,2020-12-01
2,VN,Hạ Long,precipitation,2,0.1,99.7,0.1,49600.8,2020-01-14
3,VN,Hạ Long,precipitation,4,99.1,99.1,99.1,0.0,2020-01-18
4,VN,Hạ Long,precipitation,2,2.6,2.6,2.6,0.0,2020-05-17


In [35]:
# drop column Quốc gia
df_vn_20.drop(columns=["Quốc gia"], inplace=True)

# Rename Thành phố to Province
df_vn_20.rename(columns={"Thành phố": "Province"}, inplace=True)

# Rename Phân loại to Category
df_vn_20.rename(columns={"Phân loại": "Category"}, inplace=True)

# Rename row of category from 'pressure' to 'Pressure'
df_vn_20.loc[df_vn_20["Category"] == "pressure", "Category"] = "Pressure"

# Rename row of category from 'temperature' to 'Temperature'
df_vn_20.loc[df_vn_20["Category"] == "temperature", "Category"] = "Temperature"

# Rename row of category from 'humidity' to 'Humidity'
df_vn_20.loc[df_vn_20["Category"] == "humidity", "Category"] = "Humidity"

# Rename row of category from 'wind-speed' to 'Wind Speed'
df_vn_20.loc[df_vn_20["Category"] == "wind-speed", "Category"] = "Wind Speed"

# Rename row of category from 'o3' to 'O3'
df_vn_20.loc[df_vn_20["Category"] == "o3", "Category"] = "O3"

# Rename row of category from 'co' to 'CO'
df_vn_20.loc[df_vn_20["Category"] == "co", "Category"] = "CO"

# Rename row of category from 'no2' to 'NO2'
df_vn_20.loc[df_vn_20["Category"] == "no2", "Category"] = "NO2"

# Rename row of category from 'so2' to 'SO2'
df_vn_20.loc[df_vn_20["Category"] == "so2", "Category"] = "SO2"

# Rename row of category from 'pm10' to 'PM-10'
df_vn_20.loc[df_vn_20["Category"] == "pm10", "Category"] = "PM-10"

# Rename row of category from 'pm25' to 'PM-2.5'
df_vn_20.loc[df_vn_20["Category"] == "pm25", "Category"] = "PM-2.5"

# Rename row of category from 'DEW' to 'Fog'
df_vn_20.loc[df_vn_20["Category"] == "DEW", "Category"] = "Fog"

# Rename row of category from 'precipitaion' to 'Precipitation'
df_vn_20.loc[df_vn_20["Category"] == "precipitation", "Category"] = "Precipitation"

In [41]:
# save to final-data/aqi_vietnam_2020.csv
df_vn_20.to_csv("final-data/aqi_vietnam_2020.csv", index=False)